In [1]:
import numpy as np
from random import randint
import os
import json
import settings
import pickle as pickle
from fileProcess import FileReader, FileStore 
from preProcessData import FeatureExtraction, NLP
from pyvi import ViTokenizer
from sklearn.svm import LinearSVC
from gensim import corpora, matutils
from sklearn.metrics import classification_report
from argparse import ArgumentParser
import os.path
import requests
from underthesea import ner

In [2]:
def is_valid_file(parser, arg):
    if not os.path.exists(arg):
        parser.error("The file %s does not exist!" % arg)
    else:
        return open(arg, 'r')   # return an open file handle

def readInput():
    parser = ArgumentParser(description="ikjMatrix multiplication")
    parser.add_argument("-i", dest="filename", required=True,
                        help="File need predict", metavar="FILE",
                        type=lambda x: is_valid_file(parser, x))
    args = parser.parse_args()
    return args.filename.read()
def selectPlace(list):
    for text in list:
        if text[3] == "B-LOC" or text[3] == "I-LOC":
            yield(text[0])
def findCoordinatePlace(place):
#     if 'đường' not in place:
#         return '16.059664953500032,108.21168188005686'
#     else:
        try:
            if 'đường' in place:
                place = place.replace('đường', '')
            print(place)
            data_json = requests.get('https://api.viettelmaps.com.vn:8080/gateway/searching/v1/place-api/geocoding?address='+place+'&access_token=f68f98d802ba6c0a60ee9dc44c47f48c').json()
            coordinates = ','.join(str(e) for e in data_json['data']['geometry']['coordinates'][::-1])
            return str(coordinates)
        except:
            return '16.059664953500032,108.21168188005686'

In [3]:
classifier = pickle.load(open('trained_model/logistic_model.pk','rb'))

In [26]:
# tf-idf
data = 'Tình hình dịch bệnh'
vectorizer = pickle.load(open(settings.VECTOR_EMBEDDING,'rb'))
data_features = []
data_features.append(' '.join(NLP(text=data).remove_stopwords()))
features = vectorizer.transform(data_features)

In [25]:
print(classifier.predict(features))
text_category = classifier.predict(features)[0]
print(classifier.predict_proba(features))
max(classifier.predict_proba(features)[0])

['Ngân Hàng']
[[0.00449493 0.02717287 0.21615128 0.65502868 0.00860625 0.0030003
  0.06221356 0.02333212]]


0.655028684155208

## GET COORDINATE 

# TEST

In [ ]:
data_json = requests.get('https://api.viettelmaps.com.vn:8080/gateway/searching/v1/place-api/autocomplete?input='+text_category+'&center='+coordinates+'&access_token=f68f98d802ba6c0a60ee9dc44c47f48c')


In [ ]:
data_json = data_json.json()
data_json

# TEST POS TAGGING

In [ ]:
from underthesea import ner

In [6]:
result = ner("Tìm ATM trên đường Lê Duẩn, Đà Nẵng")
result

[('Tìm', 'V', 'B-VP', 'O'),
 ('ATM', 'Np', 'B-NP', 'B-PER'),
 ('trên', 'E', 'B-PP', 'O'),
 ('đường', 'N', 'B-NP', 'B-LOC'),
 ('Lê Duẩn', 'Np', 'I-NP', 'I-LOC'),
 (',', 'CH', 'O', 'O'),
 ('Đà Nẵng', 'Np', 'B-NP', 'B-LOC')]

In [8]:
def selectPlace(list):
    for text in list:
        if text[3] == "B-LOC" or text[3] == "I-LOC":
            yield(text[0])
list_result = ' '.join(str(e) for e in selectPlace(result))           

In [10]:
data_json = requests.get('https://api.viettelmaps.com.vn:8080/gateway/placeapi/v2/place-api/geocode?address='+list_result+'&key=6ht5fdbc-1996-4f54-87gf-5664f304f3d2').json()


In [12]:
coordinates = list(data_json['results'][0]['geometry']['location'].values())[::-1]

In [13]:
coordinates

[108.20695884525776, 16.06657398864627]

In [15]:

url = "https://api.viettelmaps.vn/gateway/searching/v1/search/geojson/address?access_token=6ht5fdbc-1996-4f54-87gf-5664f304f3d2"
headers = {
  'Content-Type': 'application/json'
}
payload = {"limit":"10","offset":"0","sortByDistance":"true","query":"","proximity":"","mode":"search_result"}

In [19]:
payload['query'] = text_category
payload['proximity'] = [108.21636479342715, 16.069880066167755]

In [20]:
payload

{'limit': '10',
 'offset': '0',
 'sortByDistance': 'true',
 'query': 'ATM',
 'proximity': [108.21636479342715, 16.069880066167755],
 'mode': 'search_result'}

In [21]:
r = requests.post(url, headers=headers, data=json.dumps(payload))
print(r.text)

{"type":"FeatureCollection","query":["ATM"],"features":[{"id":"23774","poiType":21,"type":"Feature","place_type":["place"],"relevance":1.0,"text":"ATM - MaritimeBank - Lê Duẩn","place_name":"92 - 94 Lê Duẩn, Quận Hải Châu Đà Nẵng","center":[108.21622185409069,16.070774998515844],"properties":{},"geometry":{"type":"Point","coordinates":[108.21622185409069,16.070774998515844]},"imagepath":"https://files.viettelmap.vn/images/thumb/poi/atm.png","markername":"https://files.viettelmap.vn/images/marker/poi/atm.png","saved":false},{"id":"9135","poiType":21,"type":"Feature","place_type":["place"],"relevance":1.0,"text":"ATM Techcombank - Techcombank Thanh Khe","place_name":"92 - 94 Le Duan Street, Phường Thạch Thang, Quận Hải Châu, Đà Nẵng","center":[108.21644481271505,16.071067862212658],"properties":{},"geometry":{"type":"Point","coordinates":[108.21644481271505,16.071067862212658]},"imagepath":"https://files.viettelmap.vn/images/thumb/poi/atm.png","markername":"https://files.viettelmap.vn/im